# Install Packages

In [1]:
# Install Spark
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

# Let's import the libraries we will need
import itertools

# Import PySpark
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=29667ed862b1905350d98786b187a8202ec2fa3ebfaa588567ed09d09c128c5b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 11 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

# Read the Data In

Download and read the data in from

* https://course.ccs.neu.edu/cs6220/fall2023/lecture-4/id_publishers.csv
* https://course.ccs.neu.edu/cs6220/fall2023/lecture-4/id_titles.csv

Make sure you explore the data (some titles that have commas also have quotes).

In [2]:
!wget https://course.ccs.neu.edu/cs6220/fall2023/lecture-4/id_publishers.csv
!wget https://course.ccs.neu.edu/cs6220/fall2023/lecture-4/id_titles.csv

!head -n 10000 id_publishers.csv > id_pubtest1k.csv
!head -n 9081 id_titles.csv > id_titletest1k.csv

--2023-11-29 00:28:38--  https://course.ccs.neu.edu/cs6220/fall2023/lecture-4/id_publishers.csv
Resolving course.ccs.neu.edu (course.ccs.neu.edu)... 129.10.117.35
Connecting to course.ccs.neu.edu (course.ccs.neu.edu)|129.10.117.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74704360 (71M) [text/csv]
Saving to: ‘id_publishers.csv’

id_publishers.csv   100%[===================>]  71.24M  41.0MB/s    in 1.7s    

2023-11-29 00:28:40 (41.0 MB/s) - ‘id_publishers.csv’ saved [74704360/74704360]

--2023-11-29 00:28:40--  https://course.ccs.neu.edu/cs6220/fall2023/lecture-4/id_titles.csv
Resolving course.ccs.neu.edu (course.ccs.neu.edu)... 129.10.117.35
Connecting to course.ccs.neu.edu (course.ccs.neu.edu)|129.10.117.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39983488 (38M) [text/csv]
Saving to: ‘id_titles.csv’

id_titles.csv       100%[===================>]  38.13M  34.6MB/s    in 1.1s    

2023-11-29 00:28:41 (34.6 MB/s) - ‘id_t

In [3]:
#@title Read the data in

small_data = False #@param ["False", "True"] {type:"raw"}

if small_data:
  pdframe = spark.read.option("header", True).csv("id_pubtest1k.csv")
  tdframe = spark.read.option("header", True).csv("id_titletest1k.csv")
else:
  pdframe = spark.read.option("header", True).csv("id_publishers.csv")
  tdframe = spark.read.option("header", True).csv("id_titles.csv")


In [4]:
# pdframe.show()
# tdframe.show()

# Join Types

Review all of the [join types](https://www.geeksforgeeks.org/pyspark-join-types-join-two-dataframes/).

Do an inner join, an outer join, and a left join.

In [6]:
tp_inner = tdframe.join(pdframe, "ID", "inner")
tp_outer = tdframe.join(pdframe, "ID", "outer")
tp_left = tdframe.join(pdframe, "ID", "left")

tp_inner.show()
tp_outer.show()
tp_left.show()

+------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+-------------+
|    ID|               TITLE|               STORY|           PUBLISHER|CATEGORY|               STORY|            HOSTNAME|                 URL|    TIMESTAMP|
+------+--------------------+--------------------+--------------------+--------+--------------------+--------------------+--------------------+-------------+
|   100|McDonald's Not Lo...|d4lvRcSzuglGdFMmq...|      The News Ledge|       b|d4lvRcSzuglGdFMmq...|   www.newsledge.com|http://www.newsle...|1394471390131|
|  1000|Buy Titanfall And...|dw0Jnfj8vEPaLZM8T...|     Just Push Start|       t|dw0Jnfj8vEPaLZM8T...|www.justpushstart...|http://www.justpu...|1394493720128|
| 10000|Nearly one in thr...|dtBNhkt0YyqHCuM_A...|              Quartz|       t|dtBNhkt0YyqHCuM_A...|              qz.com|http://qz.com/186...|1394714666784|
|100000|Obama Nominates B...|dJ_k5DjBr5MzK0MHf...|  

In [ ]:
article_no_publisher = tp_outer.filter(tp_outer['PUBLISHER'].isNull())
article_no_title = tp_outer.filter(tp_outer['TITLE'].isNull())

article_no_publisher.show()
article_no_title.show()

+------+--------------------+--------------------+---------+--------+--------+----+---------+
|    ID|               STORY|               TITLE|PUBLISHER|CATEGORY|HOSTNAME| URL|TIMESTAMP|
+------+--------------------+--------------------+---------+--------+--------+----+---------+
|100033|dJ_k5DjBr5MzK0MHf...|Obama Turns to Se...|     null|    null|    null|null|     null|
|100077|dJ_k5DjBr5MzK0MHf...|ACA marches on, d...|     null|    null|    null|null|     null|
|100268|dM3BF5lflKhsL6MQ-...|Sen. Menendez's d...|     null|    null|    null|null|     null|
|100324|dFxU4YSThH_gU7MT9...|Vaginas, made in ...|     null|    null|    null|null|     null|
| 10039|dtBNhkt0YyqHCuM_A...|SoftBank CEO says...|     null|    null|    null|null|     null|
|100517|dBU-y8mnlizhV4Mzv...|Today's Biotech S...|     null|    null|    null|null|     null|
|100655|dwnBgdLk-3bzGBMNi...|Ebola outbreak re...|     null|    null|    null|null|     null|
|100746|dmWc0yBkvZ7rTuMUy...|Cardinal George p...|     null|